## Load CSV

In [ ]:
import pandas as pd
from transformers import BertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader, random_split

train_file = 'data/train.csv'
test_file = 'data/test.csv'
sample_file = 'data/sample.csv'

train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)
sample_df = pd.read_csv(sample_file)

train_df['text'] = train_df['text'].apply(str)
test_df['text'] = test_df['text'].apply(str)
sample_df = pd.read_csv('sample.csv')